# Reading and Exercises

## Reading - The Greeks

If $V$ is the price of an option, then we can measure the sensitivity of the price to market conditions using partial derivatives of $V$.
These partial derivatives have standard names and are collectively called "the Greeks."
$$
\begin{split}
\Delta&:=\frac{\partial V}{\partial S} \\ 
\Gamma&:=\frac{\partial^2 V}{\partial S^2} \\
\rho&:=\frac{\partial V}{\partial r} \\
\theta&:=\frac{\partial V}{\partial t} \\
\text{Vega}&:=\frac{\partial V}{\partial \sigma}
\end{split}
$$
There is no Greek letter called "Vega" so the name for this last sensitivity is rather strange!

Since these derivatives measure the sensitivity to market changes, they can be viewed as a form of risk-figure. One advantage they have as risk figures is that you can compute the Greeks of a portfolio just by adding up the Greeks of its components. This is because partial derivatives are linear.

If a trader writes (i.e. sells) a call option and replicates it using the delta-hedging strategy, then the delta of their portfolio is 0 at all times. This is because their portfolio consists of $-1$ units of the option and $\Delta$ units of the stock. So the total $\Delta$ is $-1 \cdot  \Delta + 1 \cdot \Delta=0$, since the $\Delta$ of the stock is 1 ($\frac{\partial S}{\partial S}=1$). So the $\Delta$ hedging strategy can be viewed
as the strategy of ensuring that this particular risk-measure is always 0.

In practice, derivatives traders manage their portfolios to ensure that most of the Greek's are low as, at least heuristically, one would then expect that the portfolio is hedged against risks such as a change in interest rate or volatility.

## Reading - Simplifying expressions with `sympy`

We will want to calculate the value of $\Delta$ for a call option. Let us now see how to do that in `sympy`.

We begin, as usual by creating the symbols we need and we then compute $V$ according to the Black-Scholes formula.
We introduce a variable $\tau=T-t$.
We also indicate that all our variables will be positive as this will allow more simplification rules to be applied later.

Since $N$ is a symbolic function rather than a variable, we must create it using `Function` command instead of using `symbols`.

In [1]:
from sympy import *

S, tau, K, r, sigma, D1, D2 = symbols('S tau K r sigma D1 D2',positive=True)
N = Function('N')
d1 = 1/(sigma*sqrt(tau))*(log(S/K)+(r+sigma**2/2)*tau)
d2 = d1 - sigma * sqrt(tau)
V = S*N(d1) -K*exp(-r*tau)*N(d2)
display(V)

-K*N(-sigma*sqrt(tau) + (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))*exp(-r*tau) + S*N((tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))

If we compute the partial derivative of $V$ we will obtain an expression for $\Delta$ in terms of the function $N$ and its partial derivative

In [2]:
Delta = diff(V,S)
Delta

-K*exp(-r*tau)*Subs(Derivative(N(_xi_1), _xi_1), _xi_1, -sigma*sqrt(tau) + (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))/(S*sigma*sqrt(tau)) + N((tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau))) + Subs(Derivative(N(_xi_1), _xi_1), _xi_1, (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))/(sigma*sqrt(tau))

We know the partial derivative of $N$ satisfies
$$
\frac{d N(x)}{dx}=\frac{1}{\sqrt{2 \pi}}\exp{-x^2/2}
$$
so we can simplify these derivative terms. What we want to do is take our expression for $\Delta$ and replace all partial derivatives of the form
$$
\frac{d N(*_x)}{d *_x}
$$
with
$$
\frac{1}{\sqrt{2 \pi}}e^{-\frac{*_x^2}{2}}.
$$
In these last two expressions $*_x$ could be any variable at all, but it must be the same expression each time it occurs. The label $x$ on
the pattern indicates that all the $*_x$ expressions must be identical.

You can make changes to a `sympy` expression using the `replace` function. It uses pattern matching to find expressions of a particular form and then makes an appropriate replacement. In the code below, `Wild('x')` plays exactly the same role as the $*_x$ did in our discussion above. The word `Wild` is short for Wildcard which is a standard term in pattern matching for a term that is considered to match anything. This comes from traditional card games where the Joker is a "wildcard" which you can use to represent any other card you like.

In [3]:
Delta = Delta.replace(Derivative(N(Wild('x')),Wild('x')), exp(-Wild('x')**2/2)/(sqrt(2*pi)))
Delta

-K*exp(-r*tau)*Subs(sqrt(2)*exp(-_xi_1**2/2)/(2*sqrt(pi)), _xi_1, -sigma*sqrt(tau) + (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))/(S*sigma*sqrt(tau)) + N((tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau))) + Subs(sqrt(2)*exp(-_xi_1**2/2)/(2*sqrt(pi)), _xi_1, (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))/(sigma*sqrt(tau))

We can now use the standard `sympy` function `simplify` which attempts to simplify our expression. It manaages some basic simplification, eliminating the pointless $\xi$ terms, but doesn't achieve much more than that.

In [4]:
Delta = Delta.simplify()
Delta

(-sqrt(2)*K*sqrt(tau)*exp(-(-2*sigma**2*tau + tau*(2*r + sigma**2) + log(S**2/K**2))**2/(8*sigma**2*tau))/(2*sqrt(pi)) + S*sigma*tau*N((tau*(2*r + sigma**2) + log(S**2/K**2))/(2*sigma*sqrt(tau)))*exp(r*tau) + sqrt(2)*S*sqrt(tau)*exp(r*tau)*exp(-(tau*(2*r + sigma**2) + log(S**2/K**2))**2/(8*sigma**2*tau))/(2*sqrt(pi)))*exp(-r*tau)/(S*sigma*tau)

 We should be able to simplify expressions including the exponential of a logarithms a bit more than the `simplify` function has managed automatically. When `sympy` doesn't simplify things as much as you would like, you can tell it exactly what simplifications you want to see performed. Here, I'd like it to expand all the exponentials out in full and see what cancellations occur.

In [5]:
Delta = Delta.expand()
Delta

N(r*sqrt(tau)/sigma + sigma*sqrt(tau)/2 - log(K)/(sigma*sqrt(tau)) + log(S)/(sigma*sqrt(tau)))

This is a big simplification. In fact this can be written very crisply as
$$\Delta = N(d_1).$$
We check this on the next line to make sure by simlifying $\Delta - N(d_1)$. We expect zero.

In [6]:
(N(d1)-Delta).simplify()

0

## Exercise

Use `sympy` to show that $V$ satisfies the Black-Scholes PDE
$$
\frac{\partial V}{\partial t} + \frac{1}{2} S^2 \frac{\partial^2 V}{\partial S^2 } + r S \frac{\partial V}{\partial S } - r V = 0.
$$
This may be written equivalently as 
$$
\theta + \frac{1}{2} S^2 \Gamma + r S \Delta - r V = 0.
$$

In [13]:
Gamma = diff( Delta, S )
Gamma = Gamma.replace(Derivative(N(Wild('x')),Wild('x')), exp(-Wild('x')**2/2)/(sqrt(2*pi)))
Gamma = Gamma.simplify()
theta = -diff(V, tau)
theta = theta.replace(Derivative(N(Wild('x')),Wild('x')), exp(-Wild('x')**2/2)/(sqrt(2*pi)))
theta = theta.simplify()
lhs = theta + sigma**2*S**2*Gamma/2 + r*S*Delta - r*V
lhs = lhs.simplify()
lhs.expand().simplify()


0

## Exercise

**This exercise is for students who have studied FM02 (or another, similar course, on continuous time mathematical finance) and wish to relate our results to their other studies. If you have never studied arbitrage, then you can skip this question. **

We have derived the important results of Black and Scholes theory without discussing arbitrage at all. We have focussed on *replication* rather than arbitrage. Many students (and indeed financial mathematicians) fixate upon the concept of arbitrage. I have often heard students claim that Black and Scholes theory follows "by arbitrage".

In fact, the only use of the concept of arbitrage in their theory is to prove that the Black-Scholes price is the *unique* replication price of a call option. Prove that any trading strategy that replicates a call option in the Black-Scholes model requires an wealth equal to the Black-Scholes price. You should assume that there is no arbitrage in a market consisting of a stock following geometric Brownian motion and a risk-free bank account.

### Solution

Suppose that there is another replication strategy which starts with a portfolio value of $W$ with $V \neq W$. This means that given an initial welath $W-V$ one can follow a combination of the alternative replication strategy with initial wealth $W$ and the Black-Scholes strategy with initial wealth $-V$ to hedge the position of $-1$ call options to obtain a guaranteed final payout of $0$. Hence one can start with an initial wealth of $0$ to obtain a guaranteed payoff of $e^{rT}(V-W)$ by putting $(V-W)$ into the risk-free bank account and following the two replication strategies with the starting wealth $W-V$. If $V-W>0$ this would be an arbitrage strategy. If $V-W<0$, following essentially the same strategy but with opposite signs will yield an arbitrage. So the existence of two replication strategies with different prices contradicts the assumption that there is no arbitarge.

## Exercise

**This exercise is for students who have seen a proof of Black and Scholes original result and wish to compare it with our approach.**

The original result of Black and Scholes shows that Delta hedging is exact in continous time.
The proof is very similar to the one we have given except that you should replace all the $\delta$ terms
with a $d$ and you then do not need the $\ldots$ terms in the equation.

The other important difference between our approach and the classical approach is the derivation of the SDE
$$
d \Pi_t = \Pi_t r \, d t + q_t (S_t (\mu-r) \delta t + S_t \sigma \, d W_t).
$$   
This is analogous to our difference equation
$$
\delta \Pi_t = \Pi_t r \delta t + q_t (S_t (\mu-r) \delta t + S_t \sigma \delta W_t) + \ldots
$$
which we have derived by looking at the cashflows at each time point on our grid. In the proof you have seen of Black and Scholes result,
how can is the SDE for the portfolio wealth justified?

### Solution

It is justified *by definition*! Real trading involves finite amounts of money being traded at discrete times. You can calculate the impact of trading on your portfolio by straightforward accounting. Continuous-time trading involves infinitesimal amounts of money being traded in continuous time, and so one needs to *assume* the appropriate accounting rules for these infinitesimal quantities. This is done by defining what is meant by a "self-financing trading strategy". This is tantamount to assuming that the total wealth satisfies the given SDE.